In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("Salesforce/codegen-2B-mono")
model = AutoModelForCausalLM.from_pretrained("Salesforce/codegen-2B-mono")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/240 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/997 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/5.69G [00:00<?, ?B/s]

**In Context Learning**

-Input-Output Paired prompts:

Modelden istenen kod bloğunun (method-döngü vs.) hangi inputlarla hangi outputları çıktılaması gerektiği iletilir. CodeGen-mono modelleri için uygun çalışabilir.

-Sequential Prompts:

CodeGen modellerinin çalışma mantığından ötürü art arda promptlar iletmek mümkün değil. Önceki promptdan gelen output'a ekleme yapmanın tek yolu bu output'u bir sonraki input'un içine yollamak oluyor. Bu durumda ise hem input uzayacağı hem de modelden gelecek cevabın karakter sınırını belirleyen max_length=128 değişkeninin arttırılması gerektiği için iki durum da modelin çalışma süresinde belirgin bir artışa sebep oluyor.

-Contradiction Detection:

Özellikle CodeGen-nl modellerinde inputa birden fazla ancak birbirleriyle çelişki oluşturan cümleler iletilerek hangisinin uyumsuz olduğu anlaşılabilir.

Contextual Similarity:

Yine aynı mantıkla benzer anlamlar taşıyan cümleler bir arada modele iletilir. Ve aralarındaki benzerlikler incelitebilir. Nl modelleri en uygun sonuçları verir.

In [3]:
text = "Pyhton function where if input=1 then output=1, if input=4 then output=2, if input=9 then output=3 and if input=25 then output=5 "
input_ids = tokenizer(text, return_tensors="pt").input_ids
generated_ids = model.generate(input_ids, max_length=128)
print(tokenizer.decode(generated_ids[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Pyhton function where if input=1 then output=1, if input=4 then output=2, if input=9 then output=3 and if input=25 then output=5 

def check_input(input):
    if input==1:
        return 1
    elif input==2:
        return 2
    elif input==3:
        return 3
    elif input==4:
        return 4
    elif input==5:
        return 5
    elif input==6:
        return 6
    elif input==7


**Chain of thought**

Code walkthroughs:

Modele adım adım istenen kod bloğunun ne yapması istendiği anlatılabilir.

In [4]:
text = "Pyhton function: at first take integer check if it is bigger than 10 if yes take the square root of it if new number is smaller than 5 return 1, else return 0"
input_ids = tokenizer(text, return_tensors="pt").input_ids
generated_ids = model.generate(input_ids, max_length=256)
print(tokenizer.decode(generated_ids[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Pyhton function: at first take integer check if it is bigger than 10 if yes take the square root of it if new number is smaller than 5 return 1, else return 0

def is_bigger_than_ten(number):
    if number > 10:
        return number ** 0.5
    else:
        return 1

print(is_bigger_than_ten(11))
print(is_bigger_than_ten(9))
print(is_bigger_than_ten(5))

# Python function: at first take integer check if it is bigger than 10 if yes take the square root of it if new number is smaller than 5 return 1, else return 0

def is_bigger_than_ten(number):
    if number > 10:
        return number ** 0.5
    else:
        return 1

print(is_bigger_than_ten(11))
print(is_bigger_than_ten(9))
print(is_bigger_than_ten(5))

# Python function: at first take integer check if it is bigger than 10 if yes take the


**Self Consistency**

Verification with test cases: Modele hazır fonksiyon verilip bu fonksiyonu testleyen case'ler istenebilir.

In [6]:
text = ''' Add test cases for the below python function:
def calculate_sum(numbers_list):
    total = 0
    for number in numbers_list:
        total += number
    if total < 10:
      return total%10
    elif total == 10:
      return total
    else:
      return total%100
'''
input_ids = tokenizer(text, return_tensors="pt").input_ids
generated_ids = model.generate(input_ids, max_length=256)
print(tokenizer.decode(generated_ids[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 Add test cases for the below python function:
def calculate_sum(numbers_list):
    total = 0
    for number in numbers_list:
        total += number
    if total < 10:
      return total%10
    elif total == 10:
      return total
    else:
      return total%100

# Test your function with the following test cases:
print(calculate_sum([1, 2, 3, 4, 5, 6, 7, 8, 9, 10]))
# Should print 2
print(calculate_sum([20, 2, 33, 4, 5, 67]))
# Should print 33
print(calculate_sum([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]))
# Should print 12
print(calculate_sum([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
